In [186]:
import requests
import pandas as pd
import os
import time
from dotenv import load_dotenv


# .env 파일에서 환경 변수 로드
load_dotenv()

True

# API 응답메시지 명세를 참고해 만들기

In [230]:
final_result = []
result = []
page = 1
while True:
    url = 'http://apis.data.go.kr/1160100/service/GetSmallLoanFinanceInstituteInfoService/getOrdinaryFinanceInfo'
    payload = dict(serviceKey=os.getenv('service_key'), pageNo=page, numOfRows=1000)
    r = requests.get(url, params=payload)
    print(r.status_code)
    data = r.text
    
    soup = BeautifulSoup(data, 'xml')

    
    tags = soup.find_all(True) 
    tag_dict = {tag.name: tag.get_text(strip=True) for tag in tags}
    for key, value in tag_dict.items():
        result.append(f"<{key}> : {value}")
#     final_result.append(result)

    total_pages = int(soup.find('totalCount').string) // int(soup.find('numOfRows').string) +1
    print(f"{page}/{total_pages} 수집중", end="\r")
    time.sleep(0.5)    
    
    if page < total_pages:
        page += 1
    else:
        print(end="\n")
        print("finish")
        break

200
200 수집중
200 수집중
200 수집중
4/4 수집중
finish


In [233]:
type(result)

list

In [236]:
result[27:30]

['<incm> : 3인가구 기준 중위소득 이하', '<rsdArea> : ', '<crdtSc> : 없음']

In [268]:
final_df = pd.DataFrame()
for item in result:
    temp = pd.DataFrame([item])  
    final_df = pd.concat([final_df, temp])
final_df

# final_df = pd.DataFrame()
# for item in result_dfs:
#     temp = pd.DataFrame(item)  
#     final_df = pd.concat([final_df, temp])
# final_df

,0
0,<response> : 00NORMAL SERVICE.1000136962022021...
0,<header> : 00NORMAL SERVICE.
0,<resultCode> : 00
0,<resultMsg> : NORMAL SERVICE.
0,<body> : 1000136962022021미소금융 창업자금_임차보증금대출7000...
...,...
0,<cv19SuprCtn> :
0,<cv19SuprTgtDtlCtn> :
0,<mgmDln> :
0,<prdCtg2> :


In [241]:
type(final_df)

pandas.core.frame.DataFrame

# API 응답에서 xml tag를 추출해 자동으로 데이터 정리하기

In [242]:
url = 'http://apis.data.go.kr/1160100/service/GetSmallLoanFinanceInstituteInfoService/getOrdinaryFinanceInfo'
payload = dict(serviceKey=os.getenv('service_key'), pageNo=1, numOfRows=10)
r = requests.get(url, params=payload)
print(r.status_code)
data = r.text
soup = BeautifulSoup(data, 'xml')

200


In [263]:
# 중복 방지 위해 setdefault 사용
# response ~ item까지는 필요 X
keys = {}
for tags in soup.find_all()[10:]:
    if tags.name == 'item':
        continue
    else:
#     print(tags.name)
#     temp.append(tags.name)
        keys.setdefault(f"{tags.name}", "")
# del keys['item']
# len(keys)
result2 = {}
result_dfs = []
for key in keys:
    for item in soup.find_all(key):
        result2.setdefault(key, []).append(item.string)
result_dfs.append(pd.DataFrame(result2))

In [264]:
result_dfs

[    basYm snq              finPrdNm   lnLmt irtCtg   irt maxTotLnTrm  \
 0  202202   1     미소금융 창업자금_임차보증금대출  7000만원   변동금리  4.5%          6년   
 1  202202   2  미소금융 창업자금_생계형차량 구입자금  2000만원   변동금리  4.5%          6년   
 2  202202   3   미소금융 창업자금_창업초기 운영자금  2000만원   변동금리  4.5%          6년   
 3  202202   4   미소금융 창업자금_창업초기 시설자금  2000만원   변동금리  4.5%          6년   
 4  202202   5             미소금융 운영자금  2000만원   변동금리  4.5%         5.5   
 5  202202   6           미소금융 시설개선자금  2000만원   변동금리  4.5%         5.5   
 6  202202   7           미소금융 긴급생계자금  1000만원   변동금리  4.5%          5년   
 7  202202   8           미소금융 취업성공대출   300만원   변동금리  4.5%          3년   
 8  202202   9          미소금융 교육비지원대출   500만원   변동금리  4.5%          6년   
 9  202202  10         미소금융 취약계층자립자금  1200만원   고정금리    3%          6년   
 
   maxDfrmTrm maxRdptTrm     rdptMthd  ... prdExisYn cv19Rfrc prdCtg prdNm  \
 0         1년         5년  원(리)금균등분할상환  ...         Y     None      1  대출상품   
 1         1년         5년  원(리)금균등분할상환  

In [266]:
final_df = pd.DataFrame()
for item in result_dfs:
    temp = pd.DataFrame(item)  
    final_df = pd.concat([final_df, temp])
final_df

,basYm,snq,finPrdNm,lnLmt,irtCtg,irt,maxTotLnTrm,maxDfrmTrm,maxRdptTrm,rdptMthd,...,prdExisYn,cv19Rfrc,prdCtg,prdNm,cv19SuprCtg,cv19SuprCtn,cv19SuprTgtDtlCtn,mgmDln,prdCtg2,fileWrtDt
0,202202,1,미소금융 창업자금_임차보증금대출,7000만원,변동금리,4.5%,6년,1년,5년,원(리)금균등분할상환,...,Y,None,1,대출상품,None,None,None,상시,정책자금,202202080917
1,202202,2,미소금융 창업자금_생계형차량 구입자금,2000만원,변동금리,4.5%,6년,1년,5년,원(리)금균등분할상환,...,Y,None,1,대출상품,None,None,None,상시,정책자금,202202080917
2,202202,3,미소금융 창업자금_창업초기 운영자금,2000만원,변동금리,4.5%,6년,1년,5년,원(리)금균등분할상환,...,Y,None,1,대출상품,None,None,None,상시,정책자금,202202080917
3,202202,4,미소금융 창업자금_창업초기 시설자금,2000만원,변동금리,4.5%,6년,1년,5년,원(리)금균등분할상환,...,Y,None,1,대출상품,None,None,None,상시,정책자금,202202080917
4,202202,5,미소금융 운영자금,2000만원,변동금리,4.5%,5.5,0.5,5년,원(리)금균등분할상환,...,Y,None,1,대출상품,None,None,None,상시,정책자금,202202080917
5,202202,6,미소금융 시설개선자금,2000만원,변동금리,4.5%,5.5,0.5,5년,원(리)금균등분할상환,...,Y,None,1,대출상품,None,None,None,상시,정책자금,202202080917
6,202202,7,미소금융 긴급생계자금,1000만원,변동금리,4.5%,5년,1년,4년,원(리)금균등분할상환,...,Y,None,1,대출상품,None,None,None,상시,정책자금,202202080917
7,202202,8,미소금융 취업성공대출,300만원,변동금리,4.5%,3년,0년,3년,원(리)금균등분할상환,...,Y,None,1,대출상품,None,None,None,상시,정책자금,202202080917
8,202202,9,미소금융 교육비지원대출,500만원,변동금리,4.5%,6년,1년,5년,원(리)금균등분할상환,...,Y,None,1,대출상품,None,None,None,상시,정책자금,202202080917
9,202202,10,미소금융 취약계층자립자금,1200만원,고정금리,3%,6년,1년,5년,원(리)금균등분할상환,...,Y,None,1,대출상품,None,None,None,상시,정책자금,202202080917


In [239]:
# 문자열로 나옴
data[:100]

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<response>\n    <header>\n        <resultCode>'

# beautifulSoup으로 html, xml 파싱하기

In [4]:
# 위 데이터랑 똑같은 문자열
html_doc = """<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')

print(soup.prettify())

<html>
 <head>
  <title>
   The Dormouse's story
  </title>
 </head>
 <body>
  <p class="title">
   <b>
    The Dormouse's story
   </b>
  </p>
  <p class="story">
   Once upon a time there were three little sisters; and their names were
   <a class="sister" href="http://example.com/elsie" id="link1">
    Elsie
   </a>
   ,
   <a class="sister" href="http://example.com/lacie" id="link2">
    Lacie
   </a>
   and
   <a class="sister" href="http://example.com/tillie" id="link3">
    Tillie
   </a>
   ;
and they lived at the bottom of a well.
  </p>
  <p class="story">
   ...
  </p>
 </body>
</html>



In [9]:
# 해당 태그 객체 반환
soup.title

<title>The Dormouse's story</title>

In [10]:
# 태그 이름
soup.title.name

'title'

In [11]:
# 태그명.string --> 내용 나옴
soup.title.string

"The Dormouse's story"

In [12]:
soup.p

<p class="title"><b>The Dormouse's story</b></p>

In [13]:
soup.p['class']

['title']

In [14]:
soup.a

<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>

In [15]:
# 리스트 형태로 찾아줌
soup.find_all('a')

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(data, 'xml')
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<response>
 <header>
  <resultCode>
   00
  </resultCode>
  <resultMsg>
   NORMAL SERVICE.
  </resultMsg>
 </header>
 <body>
  <numOfRows>
   10
  </numOfRows>
  <pageNo>
   1
  </pageNo>
  <totalCount>
   3696
  </totalCount>
  <items>
   <item>
    <basYm>
     202202
    </basYm>
    <snq>
     1
    </snq>
    <finPrdNm>
     미소금융 창업자금_임차보증금대출
    </finPrdNm>
    <lnLmt>
     7000만원
    </lnLmt>
    <irtCtg>
     변동금리
    </irtCtg>
    <irt>
     4.5%
    </irt>
    <maxTotLnTrm>
     6년
    </maxTotLnTrm>
    <maxDfrmTrm>
     1년
    </maxDfrmTrm>
    <maxRdptTrm>
     5년
    </maxRdptTrm>
    <rdptMthd>
     원(리)금균등분할상환
    </rdptMthd>
    <usge>
     창업
    </usge>
    <trgt>
     사업자, 금융취약계층
    </trgt>
    <instCtg>
     공공·정부기관
    </instCtg>
    <ofrInstNm>
     서민금융진흥원
    </ofrInstNm>
    <rsdAreaPamtEqltIstm>
     전국
    </rsdAreaPamtEqltIstm>
    <suprTgtDtlCond>
     개인신용평점 하위 20% 또는 차상위계층 이하 또는 근로장려금 신청 대상(수급자)인 창업(예정)자 중, 사업소요자금 

In [7]:
soup.prdNm

<prdNm>대출상품</prdNm>

In [33]:
soup.finPrdNm.string

'미소금융 창업자금_임차보증금대출'

In [34]:
soup.lnLmt.string

'7000만원'

In [55]:
lnLmt = soup.find_all('lnLmt')
lnLmt

[<lnLmt>7000만원</lnLmt>,
 <lnLmt>2000만원</lnLmt>,
 <lnLmt>2000만원</lnLmt>,
 <lnLmt>2000만원</lnLmt>,
 <lnLmt>2000만원</lnLmt>,
 <lnLmt>2000만원</lnLmt>,
 <lnLmt>1000만원</lnLmt>,
 <lnLmt>300만원</lnLmt>,
 <lnLmt>500만원</lnLmt>,
 <lnLmt>1200만원</lnLmt>]

In [57]:
finPrdNm = soup.find_all('finPrdNm')
finPrdNm

[<finPrdNm>미소금융 창업자금_임차보증금대출</finPrdNm>,
 <finPrdNm>미소금융 창업자금_생계형차량 구입자금</finPrdNm>,
 <finPrdNm>미소금융 창업자금_창업초기 운영자금</finPrdNm>,
 <finPrdNm>미소금융 창업자금_창업초기 시설자금</finPrdNm>,
 <finPrdNm>미소금융 운영자금</finPrdNm>,
 <finPrdNm>미소금융 시설개선자금</finPrdNm>,
 <finPrdNm>미소금융 긴급생계자금</finPrdNm>,
 <finPrdNm>미소금융 취업성공대출</finPrdNm>,
 <finPrdNm>미소금융 교육비지원대출</finPrdNm>,
 <finPrdNm>미소금융 취약계층자립자금</finPrdNm>]

In [58]:
finPrdNm1 = []
lnLmt1 = []
for item1, item2 in zip(finPrdNm, lnLmt):
    finPrdNm1.append(item1.string)
    lnLmt1.append(item2.string)
print(finPrdNm1)
print(lnLmt1)

['미소금융 창업자금_임차보증금대출', '미소금융 창업자금_생계형차량 구입자금', '미소금융 창업자금_창업초기 운영자금', '미소금융 창업자금_창업초기 시설자금', '미소금융 운영자금', '미소금융 시설개선자금', '미소금융 긴급생계자금', '미소금융 취업성공대출', '미소금융 교육비지원대출', '미소금융 취약계층자립자금']
['7000만원', '2000만원', '2000만원', '2000만원', '2000만원', '2000만원', '1000만원', '300만원', '500만원', '1200만원']


In [59]:
data[:500]

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<response>\n    <header>\n        <resultCode>00</resultCode>\n        <resultMsg>NORMAL SERVICE.</resultMsg>\n    </header>\n    <body>\n        <numOfRows>10</numOfRows>\n        <pageNo>1</pageNo>\n        <totalCount>3696</totalCount>\n        <items>\n            <item>\n                <basYm>202202</basYm>\n                <snq>1</snq>\n                <finPrdNm>미소금융 창업자금_임차보증금대출</finPrdNm>\n                <lnLmt>7000만원</lnLmt>\n                <i'

In [60]:
result = []
for item in finPrdNm:
    result.append(item.string)
print(result)

['미소금융 창업자금_임차보증금대출', '미소금융 창업자금_생계형차량 구입자금', '미소금융 창업자금_창업초기 운영자금', '미소금융 창업자금_창업초기 시설자금', '미소금융 운영자금', '미소금융 시설개선자금', '미소금융 긴급생계자금', '미소금융 취업성공대출', '미소금융 교육비지원대출', '미소금융 취약계층자립자금']


In [61]:
finPrdNm = [item.string for item in soup.find_all('finPrdNm')]
lnLmt = [item.string for item in soup.find_all('lnLmt')]

In [62]:
finPrdNm

['미소금융 창업자금_임차보증금대출',
 '미소금융 창업자금_생계형차량 구입자금',
 '미소금융 창업자금_창업초기 운영자금',
 '미소금융 창업자금_창업초기 시설자금',
 '미소금융 운영자금',
 '미소금융 시설개선자금',
 '미소금융 긴급생계자금',
 '미소금융 취업성공대출',
 '미소금융 교육비지원대출',
 '미소금융 취약계층자립자금']

In [63]:
lnLmt

['7000만원',
 '2000만원',
 '2000만원',
 '2000만원',
 '2000만원',
 '2000만원',
 '1000만원',
 '300만원',
 '500만원',
 '1200만원']

In [70]:
basYm = [item.string for item in soup.find_all('basYm')]
basYm

['202202',
 '202202',
 '202202',
 '202202',
 '202202',
 '202202',
 '202202',
 '202202',
 '202202',
 '202202']

In [180]:
soup

<?xml version="1.0" encoding="utf-8"?>
<response>
<header>
<resultCode>00</resultCode>
<resultMsg>NORMAL SERVICE.</resultMsg>
</header>
<body>
<numOfRows>10</numOfRows>
<pageNo>1</pageNo>
<totalCount>3696</totalCount>
<items>
<item>
<basYm>202202</basYm>
<snq>1</snq>
<finPrdNm>미소금융 창업자금_임차보증금대출</finPrdNm>
<lnLmt>7000만원</lnLmt>
<irtCtg>변동금리</irtCtg>
<irt>4.5%</irt>
<maxTotLnTrm>6년</maxTotLnTrm>
<maxDfrmTrm>1년</maxDfrmTrm>
<maxRdptTrm>5년</maxRdptTrm>
<rdptMthd>원(리)금균등분할상환</rdptMthd>
<usge>창업</usge>
<trgt>사업자, 금융취약계층</trgt>
<instCtg>공공·정부기관</instCtg>
<ofrInstNm>서민금융진흥원</ofrInstNm>
<rsdAreaPamtEqltIstm>전국</rsdAreaPamtEqltIstm>
<suprTgtDtlCond>개인신용평점 하위 20% 또는 차상위계층 이하 또는 근로장려금 신청 대상(수급자)인 창업(예정)자 중, 사업소요자금 총액 중 자기자금 비율이 50% 이상인 자</suprTgtDtlCond>
<age>미성년자 제외</age>
<incm>상환능력 심사를 위한 소득증빙 필요</incm>
<rsdArea xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:nil="true"/>
<crdtSc>개인신용평점 하위 20%</crdtSc>
<anin xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:nil="true"/>
<housHo

In [161]:
# 따로 출력해서 값이 중복됨
bslist = []
for item in soup1:
    bslist.append(item.get_text(strip=True))
print(bslist)
print()

tags = soup.find_all(True) 
tag_key = [f"<{tag.name}>" for tag in tags]
print(tag_key)

['202202', '1', '미소금융 창업자금_임차보증금대출', '7000만원', '변동금리', '4.5%', '6년', '1년', '5년', '원(리)금균등분할상환', '창업', '사업자, 금융취약계층', '공공·정부기관', '서민금융진흥원', '전국', '개인신용평점 하위 20% 또는 차상위계층 이하 또는 근로장려금 신청 대상(수급자)인 창업(예정)자 중, 사업소요자금 총액 중 자기자금 비율이 50% 이상인 자', '미성년자 제외', '상환능력 심사를 위한 소득증빙 필요', '', '개인신용평점 하위 20%', '', '', '', '', '', '', '미소금융 전국 지점 방문', '없음', '없음', '9% (지연배상금률)', '우대금리: 성실상환 여부에 따라 약정금리에서 1.0%p 인하, 가산금리: 연체 31일 이상 시 약정금리에서 1.0%p 인상', '거치금리: 1천만원 이하 대출 시 2%, 1천만원 초과 대출 시 4.5% (사업장임차보증금 이내에서 지원)', '미소금융 전국 지점', '· 서민금융콜센터 (국번없이)1397<br/>· <a href=https://www.kinfa.or.kr/customerService/centerSearch.do target=_blank><font style=color:blue;>www.kinfa.or.kr/customerService/centerSearch.do </a></font><span style=font-size: 12px;>(서금원 홈페이지 > 서민금융통합지원센터 > 전국센터찾기 > 미소지점)</span>', '<a href=https://www.kinfa.or.kr target=_blank><font style=color:blue;>www.kinfa.or.kr </a></font><span style=font-size: 12px;>(서금원 홈페이지 > 서민금융한눈에 > 창업·운영자금 > 미소금융 창업·운영자금)</span>', '사업자, 금융취약계층', '미소금융 전국 지점', 'Y', '', '1',

In [185]:
tags = soup.find_all(True) 
tag_dict = {tag.name: tag.get_text(strip=True) for tag in tags}
for key, value in tag_dict.items():
    print(f"<{key}> : {value}")

<response> : 00NORMAL SERVICE.10136962022021미소금융 창업자금_임차보증금대출7000만원변동금리4.5%6년1년5년원(리)금균등분할상환창업사업자, 금융취약계층공공·정부기관서민금융진흥원전국개인신용평점 하위 20% 또는 차상위계층 이하 또는 근로장려금 신청 대상(수급자)인 창업(예정)자 중, 사업소요자금 총액 중 자기자금 비율이 50% 이상인 자미성년자 제외상환능력 심사를 위한 소득증빙 필요개인신용평점 하위 20%미소금융 전국 지점 방문없음없음9% (지연배상금률)우대금리: 성실상환 여부에 따라 약정금리에서 1.0%p 인하, 가산금리: 연체 31일 이상 시 약정금리에서 1.0%p 인상거치금리: 1천만원 이하 대출 시 2%, 1천만원 초과 대출 시 4.5% (사업장임차보증금 이내에서 지원)미소금융 전국 지점· 서민금융콜센터 (국번없이)1397<br/>· <a href=https://www.kinfa.or.kr/customerService/centerSearch.do target=_blank><font style=color:blue;>www.kinfa.or.kr/customerService/centerSearch.do </a></font><span style=font-size: 12px;>(서금원 홈페이지 > 서민금융통합지원센터 > 전국센터찾기 > 미소지점)</span><a href=https://www.kinfa.or.kr target=_blank><font style=color:blue;>www.kinfa.or.kr </a></font><span style=font-size: 12px;>(서금원 홈페이지 > 서민금융한눈에 > 창업·운영자금 > 미소금융 창업·운영자금)</span>사업자, 금융취약계층미소금융 전국 지점Y1대출상품상시정책자금2022020809172022022미소금융 창업자금_생계형차량 구입자금2000만원변동금리4.5%6년1년5년원(리)금균등분할상환창업사업자, 금융취약계층공공·정부기관서민금융진흥원전국개인신용평점 하위 20% 

In [184]:
tag_dict['basYm']

'202202'

In [202]:
df = pd.json_normalize(tag_dict)
df

,response,header,resultCode,resultMsg,body,numOfRows,pageNo,totalCount,items,item,...,prdExisYn,cv19Rfrc,prdCtg,prdNm,cv19SuprCtg,cv19SuprCtn,cv19SuprTgtDtlCtn,mgmDln,prdCtg2,fileWrtDt
0,00NORMAL SERVICE.10136962022021미소금융 창업자금_임차보증금...,00NORMAL SERVICE.,00,NORMAL SERVICE.,10136962022021미소금융 창업자금_임차보증금대출7000만원변동금리4.5%6...,10,1,3696,2022021미소금융 창업자금_임차보증금대출7000만원변동금리4.5%6년1년5년원(...,20220210미소금융 취약계층자립자금1200만원고정금리3%6년1년5년원(리)금균등...,...,Y,,1,대출상품,,,,상시,정책자금,202202080917


In [204]:
# total_pages = int(soup.find('totalCount').string) // int(soup.find('numOfRows').string) +1